In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
import dateutil
from datetime import datetime
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE


In [3]:
dataset = pd.read_csv('ar.csv')
print(dataset.columns.values)
dataset.info()

['countryCode' 'customerID' 'PaperlessDate' 'invoiceNumber' 'InvoiceDate'
 'DueDate' 'InvoiceAmount' 'Disputed' 'SettledDate' 'PaperlessBill'
 'DaysToSettle' 'DaysLate']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2466 entries, 0 to 2465
Data columns (total 12 columns):
countryCode      2466 non-null int64
customerID       2466 non-null object
PaperlessDate    2466 non-null object
invoiceNumber    2466 non-null int64
InvoiceDate      2466 non-null object
DueDate          2466 non-null object
InvoiceAmount    2466 non-null float64
Disputed         2466 non-null object
SettledDate      2466 non-null object
PaperlessBill    2466 non-null object
DaysToSettle     2466 non-null int64
DaysLate         2466 non-null int64
dtypes: float64(1), int64(4), object(7)
memory usage: 231.3+ KB


In [4]:
#total no of invoices
totalinvoices=dataset.shape[0]
#dataset['customerID'].count()
print("Total no of invoice:",totalinvoices)

Total no of invoice: 2466


In [5]:
#No. of delayed invoices
delayed=dataset[(dataset.DaysLate>0)]
totaldelayed=delayed.shape[0]
print("No of delayed invoices:",totaldelayed)

No of delayed invoices: 877


In [6]:
percentofdelayed=(totaldelayed/totalinvoices)*100
print("percentage of delayed invoice:",percentofdelayed)

percentage of delayed invoice: 35.56366585563666


In [7]:
#No of payers in the data set
#totalpayers=len(dataset.groupby('customerID'))
#print(dataset['customerID'].value_counts())
totalpayers=(dataset['customerID'].nunique())
print("Total no of payers:",totalpayers)

Total no of payers: 100


In [8]:
# max delayed days
print("max delayed days:",dataset['DaysLate'].max())

max delayed days: 45


In [9]:
#No of invoice per payer
grouped=dataset.groupby('customerID',as_index=False)
invoice_count=grouped.agg({"invoiceNumber":"count"})
invoice_count.columns=['customerID','total']
invoice_count_sort=invoice_count.sort_values('total', ascending=False)
print(invoice_count.shape)
print(invoice_count.columns.values)
print(invoice_count_sort.head(5))

#my.plot()
#plt.show()

(100, 2)
['customerID' 'total']
    customerID  total
85  9149-MATVB     36
80  8887-NCUZC     35
34  4640-FGEJI     35
4   0688-XNJRO     34
90  9286-VLKMI     34


In [10]:
# delayed invoices per payer
delayed=delayed.groupby('customerID',as_index=False)
delayed_invoice_count=delayed.agg({'invoiceNumber':'count'})
#right = pd.DataFrame(mydelay)
delayed_invoice_count.columns=['customerID','delayed']

print(delayed_invoice_count.shape)
delayed_invoice_count_sort=delayed_invoice_count.sort_values('customerID')
print(delayed_invoice_count.columns.values)
print(delayed_invoice_count_sort.head(5))
#mydelay.plot()
#plt.show()

(83, 2)
['customerID' 'delayed']
   customerID  delayed
0  0379-NEVHP        1
1  0465-DTULQ       14
2  0625-TNJFG        5
3  0688-XNJRO       32
4  0706-NRGUP       10


In [11]:
delayed_days_avg=delayed.agg({'DaysLate':'mean'})
delayed_days_avg.columns=['customerID','avgDaysDelayed']
print(delayed_days_avg.head(5))


   customerID  avgDaysDelayed
0  0379-NEVHP       17.000000
1  0465-DTULQ        9.857143
2  0625-TNJFG        4.000000
3  0688-XNJRO       15.343750
4  0706-NRGUP        6.900000


In [12]:
settled_days_avg=grouped.agg({'DaysToSettle':'mean'})
settled_days_avg.columns=['customerID','avgDaysToSettle']
print(settled_days_avg.head(5))

   customerID  avgDaysToSettle
0  0187-ERLSR        12.937500
1  0379-NEVHP        17.444444
2  0465-DTULQ        33.730769
3  0625-TNJFG        24.750000
4  0688-XNJRO        44.382353


In [13]:
# delay ratio per payer
invoice_count_stats=pd.merge(invoice_count,delayed_invoice_count,on='customerID',how='left').fillna(0)
invoice_count_stats=invoice_count_stats.sort_values('customerID')
invoice_count_stats['paid']=invoice_count_stats['total']-invoice_count_stats['delayed']
invoice_count_stats['delayRatio']=(invoice_count_stats['delayed']/invoice_count_stats['total'])
print(invoice_count_stats.head(10))

   customerID  total  delayed  paid  delayRatio
0  0187-ERLSR     16      0.0  16.0    0.000000
1  0379-NEVHP     27      1.0  26.0    0.037037
2  0465-DTULQ     26     14.0  12.0    0.538462
3  0625-TNJFG     28      5.0  23.0    0.178571
4  0688-XNJRO     34     32.0   2.0    0.941176
5  0706-NRGUP     18     10.0   8.0    0.555556
6  0709-LZRJV     25     11.0  14.0    0.440000
7  0783-PEPYR     21     20.0   1.0    0.952381
8  1080-NDGAE     31     18.0  13.0    0.580645
9  1168-BEASA     23      2.0  21.0    0.086957


In [14]:
# total paid invoice amount per payer
paid_tot=grouped.agg({"InvoiceAmount":"sum"})
print(paid_tot.columns.values)
paid_tot.columns=['customerID','totalAmt']
print(paid_tot.head(5))

['customerID' 'InvoiceAmount']
   customerID  totalAmt
0  0187-ERLSR   1072.63
1  0379-NEVHP   1584.18
2  0465-DTULQ   1360.12
3  0625-TNJFG   1627.26
4  0688-XNJRO   1231.45


In [15]:
# total delayed invoice amount per payer
delayed_tot=delayed.agg({"InvoiceAmount":"sum"})
delayed_tot.columns=['customerID','delayedAmt']
print(delayed_tot.head(5))

   customerID  delayedAmt
0  0379-NEVHP       48.65
1  0465-DTULQ      670.39
2  0625-TNJFG      301.94
3  0688-XNJRO     1181.00
4  0706-NRGUP      325.42


In [16]:
invoice_amt_stats=pd.merge(paid_tot,delayed_tot,on='customerID',how='left').fillna(0)
invoice_amt_stats['paidAmt']=invoice_amt_stats['totalAmt']-invoice_amt_stats['delayedAmt']
invoice_amt_stats['delayAmtRatio']=(invoice_amt_stats['delayedAmt']/invoice_amt_stats['totalAmt'])
print(invoice_amt_stats.head(5))

   customerID  totalAmt  delayedAmt  paidAmt  delayAmtRatio
0  0187-ERLSR   1072.63        0.00  1072.63       0.000000
1  0379-NEVHP   1584.18       48.65  1535.53       0.030710
2  0465-DTULQ   1360.12      670.39   689.73       0.492890
3  0625-TNJFG   1627.26      301.94  1325.32       0.185551
4  0688-XNJRO   1231.45     1181.00    50.45       0.959032


In [17]:
payer_stats=pd.merge(invoice_count_stats,invoice_amt_stats,on="customerID",how='left')
payer_stats=pd.merge(payer_stats,delayed_days_avg,on="customerID",how="left").fillna(0)
payer_stats=pd.merge(payer_stats,settled_days_avg,on="customerID",how="left").fillna(0)

print(payer_stats.head(2))


   customerID  total  delayed  paid  delayRatio  totalAmt  delayedAmt  \
0  0187-ERLSR     16      0.0  16.0    0.000000   1072.63        0.00   
1  0379-NEVHP     27      1.0  26.0    0.037037   1584.18       48.65   

   paidAmt  delayAmtRatio  avgDaysDelayed  avgDaysToSettle  
0  1072.63        0.00000             0.0        12.937500  
1  1535.53        0.03071            17.0        17.444444  


In [18]:

dataset['invoicemonth']=pd.to_datetime(dataset['InvoiceDate']).dt.month
dataset['invoicedate']=pd.to_datetime(dataset['InvoiceDate']).dt.day
dataset['invoiceday']=pd.to_datetime(dataset['InvoiceDate']).dt.weekday
dataset['monthend']=np.where(dataset['invoicedate']>27,1,0)
dataset['firsthalfmonth']=np.where(dataset['invoicedate']<16,1,0)
paperless={'Paper':0,'Electronic':1}
dataset['paperless']=dataset['PaperlessBill'].map(paperless)
disputed={'Yes':1,'No':0}
dataset['disputed']=dataset['Disputed'].map(disputed)



#print(dt.year, dt.month, dt.day)
print(dataset.head(1))



   countryCode  customerID PaperlessDate  invoiceNumber InvoiceDate   DueDate  \
0          391  0379-NEVHP      4/6/2013         611365    1/2/2013  2/1/2013   

   InvoiceAmount Disputed SettledDate PaperlessBill  DaysToSettle  DaysLate  \
0          55.94       No   1/15/2013         Paper            13         0   

   invoicemonth  invoicedate  invoiceday  monthend  firsthalfmonth  paperless  \
0             1            2           2         0               1          0   

   disputed  
0         0  


In [19]:
#dataset.columns.values[1]='payer'
#dataset_new=dataset.join(payer_stats.set_index('payer'))
dataset_new=pd.merge(dataset,payer_stats,on='customerID',how='left').fillna(0)
#dataset_new=dataset_new[dataset_new['customerID']=='9149-MATVB']
print(dataset_new.shape)
custlist=payer_stats['customerID'].tolist()
cat = {x: custlist.index(x) for x in custlist}
dataset_new['cust']= dataset_new['customerID'].map(cat)

## features used for training
dataset_new=dataset_new[['cust','InvoiceAmount','invoicemonth','monthend','paperless','avgDaysToSettle','disputed','DaysToSettle']]
                         #'total','totalAmt','avgDaysToSettle','DaysToSettle']]
                            # 'avgDaysToSettle'
print(dataset_new.columns.values)
#print(dataset_new.shape)
#dummies = pd.get_dummies(dataset_new.customerID)
print(dataset_new.head(2))




(2466, 29)
['cust' 'InvoiceAmount' 'invoicemonth' 'monthend' 'paperless'
 'avgDaysToSettle' 'disputed' 'DaysToSettle']
   cust  InvoiceAmount  invoicemonth  monthend  paperless  avgDaysToSettle  \
0     1          55.94             1         0          0        17.444444   
1    82          61.74             1         0          1        25.777778   

   disputed  DaysToSettle  
0         0            13  
1         1            36  


In [20]:
#dataset.plot(x='monthend', y='DaysToSettle')  
#plt.title('monthend vs DaysToSettle')  
#plt.xlabel('monthend ')  
#plt.ylabel('DaysToSettle')  
#plt.show() 

In [21]:
#dataset_new.hist(bins=50, figsize=(20,15))
#plt.savefig("attribute_histogram_plots")
#plt.show()

In [22]:
cols=dataset_new.columns
dataset_new[cols] = dataset_new[cols].apply(pd.to_numeric)
#print(dataset_new.dtypes)

In [23]:
array=dataset_new.values
n=len(dataset_new.columns)
X=array[:,0:n-1]
Y=array[:,n-1]
seed=7
X_train, X_rest, Y_train, Y_rest = model_selection.train_test_split(X,Y,test_size=0.40,random_state=seed)
X_validation,X_test,Y_validation,Y_test=model_selection.train_test_split(X_rest,Y_rest,test_size=0.50,random_state=seed)

In [24]:
# np.argwhere(np.isinf(X_train))


In [25]:
lm=LinearRegression()
#lm.fit(X_train,Y_train)

In [26]:
#predict_train=lm.predict(X_train)
#predict_val=lm.predict(X_validation)
# recursive feature elimination
rfe = RFE(lm, n-2)
fit = rfe.fit(X_train, Y_train)
print("Num Features: %d" % fit.n_features_)
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % fit.ranking_)

Num Features: 6
Selected Features: [ True False  True  True  True  True  True]
Feature Ranking: [1 2 1 1 1 1 1]


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


In [32]:
lm.fit(X_train,Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [33]:
# %  of changes in Y that can be  explained using X
print('Linear Regression R squared": %.4f' % lm.score(X_validation, Y_validation))


Linear Regression R squared": 0.7697


In [34]:
#lm.score(X_train, Y_train)

In [35]:
from sklearn.metrics import mean_squared_error

y_predict = lm.predict(X_validation)
print(y_predict[1])

regression_model_mse = mean_squared_error(y_predict, Y_validation)

regression_model_mse

27.172608892790812


34.820875531175275

In [36]:
import math

lm_rmse=math.sqrt(regression_model_mse)
print('Linear Regression RMSE: %.4f' % lm_rmse)

Linear Regression RMSE: 5.9009


In [37]:
# using random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
forest_reg = RandomForestRegressor(random_state=42)
forest_reg.fit(X_train, Y_train)
print('Random Forest R squared": %.4f' % forest_reg.score(X_validation, Y_validation))
#print(regr.predict(X_validation[0]))

Random Forest R squared": 0.7762


In [38]:
y_pred = forest_reg.predict(X_validation)
forest_mse = mean_squared_error(y_pred, Y_validation)
forest_rmse = np.sqrt(forest_mse)
print('Random Forest RMSE: %.4f' % forest_rmse)

Random Forest RMSE: 5.8170


In [39]:
# gradient boosting
from sklearn import ensemble
from sklearn.ensemble import GradientBoostingRegressor
model = ensemble.GradientBoostingRegressor()
model.fit(X_train, Y_train)
print('Gradient Boosting R squared": %.4f' % model.score(X_validation, Y_validation))
y_pred = model.predict(X_validation)
model_mse = mean_squared_error(y_pred, Y_validation)
model_rmse = np.sqrt(model_mse)
print('Gradient Boosting RMSE: %.4f' % model_rmse)


Gradient Boosting R squared": 0.7882
Gradient Boosting RMSE: 5.6586


In [40]:
test_pred_df = pd.DataFrame( { 'actual': Y_validation,
                            'predicted': np.round( y_pred, 2),
                            'residuals': Y_validation - y_pred } )
#test_pred_df[test_pred_df['residuals']<5]
test_pred_df[30:40]

,actual,predicted,residuals
30,13.0,21.94,-8.937916
31,28.0,33.95,-5.953512
32,24.0,23.59,0.413180
33,22.0,20.74,1.258330
34,23.0,21.38,1.621820
35,27.0,25.97,1.030480
36,22.0,18.74,3.260836
37,16.0,20.73,-4.730669
38,18.0,23.05,-5.054093
39,29.0,37.61,-8.608048


In [41]:
import pickle

filename = 'finalized_model.sav'
pickle.dump(lm, open(filename, 'wb'))

In [42]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)

0.733856489312197
